# Model Evaluation

## Imoprt libraries

In [460]:
import numpy as np
import pandas as pd
from dataclasses import dataclass, field
from enum import Enum
from typing import List
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Settings

In [461]:
SETTING_RESULT_PARENT_DIRECTORY_PATH = os.path.join("Result")
SETTING_TEST_DATA_FRAME_PATH = os.path.join("test_data_frame.csv")
SETTING_REVENUE_TEST_DATA_FRAME_PATH = os.path.join(
    os.pardir, "datasets", "revenue_test_data_by_date_store.csv"
)
SETTING_STATES = [
    "CA_1",
    "CA_2",
    "CA_3",
    "CA_4",
    "TX_1",
    "TX_2",
    "TX_3",
    "WI_1",
    "WI_2",
    "WI_3",
]
SETTING_FIRST_FORECAST_DAY = 1914

## Enums

In [462]:
class Metric(Enum):
    MASE = 0  # Mean Absolute Scaled Error
    RMSSE = 1  # Root Mean Squared Scaled Error
    WMSSE = 2  # Weighted Root Mean Squared Scaled Error

## Classes

In [463]:
@dataclass
class Data_Frame:
    name: str
    data_frame: pd.DataFrame


@dataclass
class MetricResult:
    data_frame_name: str
    metric: Metric
    value: float


@dataclass
class Result:
    author: str
    data_frame_list: List[Data_Frame] = field(default_factory=list)
    metric_result_list: List[MetricResult] = field(default_factory=list)

## Metric formulas

### Mean Absolute Scaled Error

In [464]:
def mase(train: np.ndarray, actual: np.ndarray, forecast: np.ndarray) -> float:
    """
    Compute Mean Absolute Scaled Error (MASE) based on the correct formula.

    Args:
        train (np.ndarray): Array of historical (training) data
        actual (np.ndarray): Array of actual values for the test period
        forecast (np.ndarray): Array of forecasted values

    Returns:
        float: MASE value
    """

    # Compute scale denominator
    denominator = np.mean(np.abs(np.diff(train)))

    if denominator == 0:
        return np.nan  # Avoid division by zero

    # Compute numerator
    numerator = np.mean(np.abs(actual - forecast))

    return np.sqrt(numerator / denominator)

### Root Mean Squared Scaled Error

In [465]:
def rmsse(train: np.ndarray, actual: np.ndarray, forecast: np.ndarray) -> float:
    """
    Compute Root Mean Squared Scaled Error (RMSSE) based on the correct formula.

    Parameters:
        train (np.ndarray): Array of historical (training) data
        actual (np.ndarray): Array of actual values for the test period
        forecast (np.ndarray): Array of forecasted values

    Returns:
        float: RMSSE value
    """

    # Compute scale denominator
    denominator = np.mean(np.square(np.diff(train)))

    if denominator == 0:
        return np.nan  # Avoid division by zero

    # Compute numerator
    numerator = np.mean(np.square(actual - forecast))

    return np.sqrt(numerator / denominator)

## Helper Functions

### Create test data frame

In [466]:
def create_test_data_frame() -> pd.DataFrame:
    if os.path.isfile(SETTING_TEST_DATA_FRAME_PATH):
        print(f"Test data frame already exists at {SETTING_TEST_DATA_FRAME_PATH}")
        test_df = pd.read_csv(SETTING_TEST_DATA_FRAME_PATH)
        return test_df.copy()

    if not os.path.isfile(SETTING_REVENUE_TEST_DATA_FRAME_PATH):
        raise FileNotFoundError(
            f"Revenue test data frame not found at {SETTING_REVENUE_TEST_DATA_FRAME_PATH}."
        )

    revenue_test_df = pd.read_csv(SETTING_REVENUE_TEST_DATA_FRAME_PATH)

    pivoted_revenue_test_df = revenue_test_df.pivot(
        index=["date", "d", "weekday"], columns="store_id", values="revenue"
    ).reset_index()

    pivoted_revenue_test_df.insert(0, "id", [(i + 1) for i in range(1941)])

    pivoted_revenue_test_df.to_csv(SETTING_TEST_DATA_FRAME_PATH, index=False)

    return pivoted_revenue_test_df.copy()

### Read results

In [467]:
def read_result(parent_directory: str) -> List[Result]:
    # Check if the given path exists and is a directory
    if not os.path.isdir(parent_directory):
        raise NotADirectoryError(f"Directory '{parent_directory}' is not found.")

    # Get a list of all subdirectories within the parent directory
    member_subdirectories = [
        d
        for d in os.listdir(parent_directory)
        if os.path.isdir(os.path.join(parent_directory, d))
    ]

    # Check if there are any subdirectories; if not, raise an error
    if not member_subdirectories:
        raise NotADirectoryError("Author subdirectories are not found.")

    result_list: List[Result] = []

    for member in member_subdirectories:
        result = Result(author=member, data_frame_list=[], metric_result_list=[])

        member_path = os.path.join(parent_directory, member)

        csv_files = [
            f
            for f in os.listdir(member_path)
            if f.endswith(".csv") and os.path.isfile(os.path.join(member_path, f))
        ]

        for csv_file in csv_files:
            data_frame = Data_Frame(name=csv_file, data_frame=pd.read_csv(csv_file))
            result.data_frame_list.append(data_frame)

        result_list.append(result)

    return result_list

### Evaluate results

In [468]:
def evaluate(
    evaluated_data_frame: pd.DataFrame,
    test_data_frame: pd.DataFrame,
    metric: Metric,
    weight: np.ndarray = None,
) -> float:
    train = test_data_frame[test_data_frame["id"] < SETTING_FIRST_FORECAST_DAY]
    actual = test_data_frame[test_data_frame["id"] >= SETTING_FIRST_FORECAST_DAY]

    match metric:
        case Metric.MASE:
            value = np.mean(
                np.array(
                    [
                        mase(
                            train=train[state].to_numpy(),
                            actual=actual[state].to_numpy(),
                            forecast=evaluated_data_frame[state].to_numpy(),
                        )
                        for state in SETTING_STATES
                    ]
                )
            )

        case Metric.RMSSE:
            value = np.mean(
                np.array(
                    [
                        rmsse(
                            train=train[state].to_numpy(),
                            actual=actual[state].to_numpy(),
                            forecast=evaluated_data_frame[state].to_numpy(),
                        )
                        for state in SETTING_STATES
                    ]
                )
            )

        case Metric.WMSSE:
            if weight is None:
                raise ValueError("'weight' must be an 1D array.")

            value = np.sum(
                weight
                * np.array(
                    [
                        rmsse(
                            train=train[state].to_numpy(),
                            actual=actual[state].to_numpy(),
                            forecast=evaluated_data_frame[state].to_numpy(),
                        )
                        for state in SETTING_STATES
                    ]
                )
            )

    return value


def evaluate_result(
    result_list: List[Result],
    test_data_frame: pd.DataFrame,
    metric: Metric,
    weight: np.ndarray = None,
):
    for result in result_list:
        result.metric_result_list.extend(
            [
                MetricResult(
                    data_frame_name=data_frame.name,
                    metric=metric,
                    value=evaluate(
                        data_frame.data_frame, test_data_frame, metric, weight
                    ),
                )
                for data_frame in result.data_frame_list
            ]
        )

### Revenue by year

In [469]:
def revenue_by_year(data_frame: pd.DataFrame, year: int = None):
    if "date" not in data_frame.columns:
        raise ValueError(
            "Invalid argument: The DataFrame must contain a 'date' column."
        )

    data_frame = data_frame.copy()

    # Extract the year from the 'date' column and add it as a new column
    data_frame["year"] = pd.to_datetime(data_frame["date"]).dt.year

    # Drop some columns
    data_frame = data_frame.drop(columns=["id", "date", "d", "weekday"])

    if year:
        min_year = data_frame["year"].min()
        max_year = data_frame["year"].max()
        if year < min_year or year > max_year:
            print(
                f"Invalid argument: Year {year} is out of range. Range: {min_year} - {max_year}."
            )
        else:
            data_frame = data_frame[data_frame["year"] == year]

    # Group by the year and sum the revenue
    revenue = data_frame.groupby("year").sum().reset_index()

    return revenue

## Main Flow

### Create train data frame

In [470]:
train_df = create_test_data_frame()
train_df = train_df[train_df["id"] < SETTING_FIRST_FORECAST_DAY]

Test data frame already exists at test_data_frame.csv


In [471]:
train_df

,id,date,d,weekday,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
0,1,2011-01-29,d_1,Saturday,10933.16,9101.52,11679.83,4561.59,6586.68,9915.78,7597.99,6454.72,5451.46,9367.88
1,2,2011-01-30,d_2,Sunday,9787.06,8417.53,12161.46,4681.41,6610.60,9804.54,7356.54,5645.77,4636.86,9868.80
2,3,2011-01-31,d_3,Monday,7201.38,5320.51,9123.86,3637.98,4551.97,6651.16,5406.70,3640.12,4621.58,7551.65
3,4,2011-02-01,d_4,Tuesday,7407.74,5550.56,10249.78,3708.92,5374.39,6985.60,5597.97,2949.96,5754.75,7181.53
4,5,2011-02-02,d_5,Wednesday,6566.12,5229.72,9538.65,3841.14,4347.07,6039.05,4069.74,2.96,2679.19,4646.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,1909,2016-04-20,d_1909,Wednesday,12347.85,11564.22,18113.40,7978.28,8950.16,11278.96,11092.53,9949.23,12212.21,9341.89
1909,1910,2016-04-21,d_1910,Thursday,11687.99,10746.54,16230.51,8169.58,8260.00,11409.73,11535.43,10361.39,12796.87,9212.23
1910,1911,2016-04-22,d_1911,Friday,14205.08,14654.14,18491.01,8301.77,9499.88,12494.57,11889.90,12292.56,14251.83,10959.56
1911,1912,2016-04-23,d_1912,Saturday,18317.93,19846.12,24861.53,9911.55,11373.50,14575.46,13093.63,14332.14,15761.02,13120.79


### Create test data frame

In [472]:
test_df = create_test_data_frame()

Test data frame already exists at test_data_frame.csv


In [473]:
test_df

,id,date,d,weekday,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
0,1,2011-01-29,d_1,Saturday,10933.16,9101.52,11679.83,4561.59,6586.68,9915.78,7597.99,6454.72,5451.46,9367.88
1,2,2011-01-30,d_2,Sunday,9787.06,8417.53,12161.46,4681.41,6610.60,9804.54,7356.54,5645.77,4636.86,9868.80
2,3,2011-01-31,d_3,Monday,7201.38,5320.51,9123.86,3637.98,4551.97,6651.16,5406.70,3640.12,4621.58,7551.65
3,4,2011-02-01,d_4,Tuesday,7407.74,5550.56,10249.78,3708.92,5374.39,6985.60,5597.97,2949.96,5754.75,7181.53
4,5,2011-02-02,d_5,Wednesday,6566.12,5229.72,9538.65,3841.14,4347.07,6039.05,4069.74,2.96,2679.19,4646.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,1937,2016-05-18,d_1937,Wednesday,12920.62,12766.25,17780.83,8116.41,9851.97,11121.03,12078.07,9605.89,12928.75,9163.29
1937,1938,2016-05-19,d_1938,Thursday,13259.90,13432.94,18635.35,8367.57,8403.09,10474.33,11196.76,10478.86,13547.39,9660.13
1938,1939,2016-05-20,d_1939,Friday,13999.65,15545.28,18219.23,8960.17,11296.88,13832.01,14667.00,11358.75,14139.33,11982.37
1939,1940,2016-05-21,d_1940,Saturday,18637.70,24088.59,23849.52,9768.51,13137.35,15212.81,15696.64,14614.05,15020.25,12370.23


### Read results

In [474]:
result_list = read_result(SETTING_RESULT_PARENT_DIRECTORY_PATH)

In [475]:
if not result_list:
    raise FileNotFoundError("No data frame found.")

### Create weight values

In [476]:
revenue_by_year_df = revenue_by_year(train_df)
revenue_by_year_df

,year,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
0,2011,2922245.86,2411535.13,4174686.87,1505776.15,2086062.89,2864113.41,2256005.47,1474474.34,1431217.15,2765218.81
1,2012,3917812.66,3063930.96,5652875.00,2061955.19,2819059.15,3902965.25,2934446.29,2017060.97,2642317.48,3636777.89
2,2013,4342793.28,3298343.92,6158423.62,2341768.79,2975205.37,4169267.01,3091774.77,2818384.70,3628699.84,3098712.01
3,2014,4645767.19,3131040.08,6744344.24,2565132.23,3153808.79,3805537.79,3635085.31,3362263.61,3957147.69,2861786.26
4,2015,5058086.52,3906887.64,7054105.72,2782280.73,3535360.24,4290980.97,4395218.56,3789069.25,4298560.96,3305906.02
5,2016,1633546.87,1597706.95,2324284.25,950233.80,1159665.62,1485372.69,1484197.36,1307982.15,1732795.18,1258505.12


In [477]:
revenue_by_year_df_2011_2015 = revenue_by_year_df[revenue_by_year_df["year"] < 2016]
revenue_by_year_df_2011 = revenue_by_year_df[revenue_by_year_df["year"] == 2011]
revenue_by_year_df_2012 = revenue_by_year_df[revenue_by_year_df["year"] == 2012]
revenue_by_year_df_2013 = revenue_by_year_df[revenue_by_year_df["year"] == 2013]
revenue_by_year_df_2014 = revenue_by_year_df[revenue_by_year_df["year"] == 2014]
revenue_by_year_df_2015 = revenue_by_year_df[revenue_by_year_df["year"] == 2015]

In [478]:
fig = px.line(revenue_by_year_df_2011_2015, x="year", y=revenue_by_year_df.columns)
fig.show()

In [479]:
rows, cols = 1, 5

fig = make_subplots(
    rows=rows,
    cols=cols,
    subplot_titles=[
        f"Revenue in {i}" for i in range(2011, 2016)
    ],  # Auto-generate titles
    specs=[
        [{"type": "pie"} for _ in range(cols)] for _ in range(rows)
    ],  # Generate pie types
)

count = 0
for year in range(2011, 2016):
    df = revenue_by_year_df.drop(columns="year")[
        revenue_by_year_df["year"] == year
    ].melt(var_name="state", value_name="revenue")

    fig.add_trace(
        go.Pie(
            labels=df["state"],  # Categories
            values=df["revenue"],  # Corresponding values
            domain=dict(x=[0, 1], y=[0, 1]),  # Expands pie size
        ),
        row=1,
        col=1 + count,
    )
    count += 1

fig.update_layout(
    showlegend=True,  # Hide legends for a cleaner look
    margin=dict(l=10, r=10, t=50, b=10),  # Reduce margins
)

fig.show()

In [480]:
revenue_by_year_df_2011_2015

,year,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
0,2011,2922245.86,2411535.13,4174686.87,1505776.15,2086062.89,2864113.41,2256005.47,1474474.34,1431217.15,2765218.81
1,2012,3917812.66,3063930.96,5652875.00,2061955.19,2819059.15,3902965.25,2934446.29,2017060.97,2642317.48,3636777.89
2,2013,4342793.28,3298343.92,6158423.62,2341768.79,2975205.37,4169267.01,3091774.77,2818384.70,3628699.84,3098712.01
3,2014,4645767.19,3131040.08,6744344.24,2565132.23,3153808.79,3805537.79,3635085.31,3362263.61,3957147.69,2861786.26
4,2015,5058086.52,3906887.64,7054105.72,2782280.73,3535360.24,4290980.97,4395218.56,3789069.25,4298560.96,3305906.02


In [481]:
# Create a copy of the original DataFrame with only the 'year' column
revenue_by_year_df_2011_2015_percentage = revenue_by_year_df_2011_2015[["year"]].copy()

# Compute total revenue for each year
total_revenue = revenue_by_year_df_2011_2015.iloc[:, 1:].sum(axis=1)

# Calculate percentage for each state efficiently
state_columns = revenue_by_year_df_2011_2015.columns[1:]
revenue_by_year_df_2011_2015_percentage[state_columns + "_percentage"] = (
    revenue_by_year_df_2011_2015[state_columns].div(total_revenue, axis=0) * 100
)

# Display the result
revenue_by_year_df_2011_2015_percentage

,year,CA_1_percentage,CA_2_percentage,CA_3_percentage,CA_4_percentage,TX_1_percentage,TX_2_percentage,TX_3_percentage,WI_1_percentage,WI_2_percentage,WI_3_percentage
0,2011,12.231404,10.093764,17.473643,6.302603,8.731462,11.988084,9.442777,6.171586,5.990528,11.574149
1,2012,11.999720,9.384398,17.313977,6.315484,8.634389,11.954244,8.987804,6.177980,8.093054,11.138949
2,2013,12.089046,9.181610,17.143222,6.518789,8.282088,11.606001,8.606583,7.845546,10.101222,8.625894
3,2014,12.270292,8.269630,17.813004,6.774967,8.329766,10.051097,9.600902,8.880332,10.451526,7.558483
4,2015,11.924821,9.210783,16.630587,6.559437,8.334879,10.116312,10.362060,8.933017,10.134182,7.793923


In [482]:
revenue_by_year_df_2011_2015_weight = revenue_by_year_df_2011_2015_percentage.drop(
    columns="year"
).sum(axis=0)
total_revenue = revenue_by_year_df_2011_2015_weight.sum()
revenue_by_year_df_2011_2015_weight = (
    revenue_by_year_df_2011_2015_weight / total_revenue
).to_numpy()
revenue_by_year_df_2011_2015_weight

array([0.12103057, 0.09228037, 0.17274887, 0.06494256, 0.08462517,
       0.11143148, 0.09400025, 0.07601692, 0.08954102, 0.0933828 ])

### Evaluate models

#### Metric: Mean Absolute Scaled Error

In [483]:
evaluate_result(result_list=result_list, test_data_frame=test_df, metric=Metric.MASE)

#### Metric: Root Mean Squared Scaled Error

In [484]:
evaluate_result(result_list=result_list, test_data_frame=test_df, metric=Metric.RMSSE)

#### Metric: Weighted Root Mean Squared Scaled Error

In [485]:
evaluate_result(
    result_list=result_list,
    test_data_frame=test_df,
    metric=Metric.WMSSE,
    weight=revenue_by_year_df_2011_2015_weight,
)

In [486]:
result_list

[Result(author='Duc', data_frame_list=[], metric_result_list=[]),
 Result(author='Huong', data_frame_list=[], metric_result_list=[]),
 Result(author='Phuong', data_frame_list=[], metric_result_list=[]),
 Result(author='Trung', data_frame_list=[Data_Frame(name='revenue_sample_data.csv', data_frame=      id        date       d   CA_1   CA_2   CA_3   CA_4   TX_1   TX_2   TX_3  \
 0   1914  2016-04-25  d_1914  14361  14361  14361  14361  14361  14361  14361   
 1   1915  2016-04-26  d_1915   5533   5533   5533   5533   5533   5533   5533   
 2   1916  2016-04-27  d_1916  12026  12026  12026  12026  12026  12026  12026   
 3   1917  2016-04-28  d_1917  12906  12906  12906  12906  12906  12906  12906   
 4   1918  2016-04-29  d_1918  14471  14471  14471  14471  14471  14471  14471   
 5   1919  2016-04-30  d_1919   5243   5243   5243   5243   5243   5243   5243   
 6   1920  2016-05-01  d_1920   8376   8376   8376   8376   8376   8376   8376   
 7   1921  2016-05-02  d_1921  12578  12578  12